# STOP THE COUNT!

Let us count only the votes emitted before 18:00.

In [55]:
data = dd.read_csv("data/votes_*.csv",
                   usecols=["timestamp", "region", "vote"],
                   parse_dates=["timestamp"],
                   dtype={"region": "category", "vote": "category"})
data["result"] = 1

In [63]:
count_per_region_before_17 = (
    data[data.timestamp.dt.hour < 17]
    .groupby(["region", "vote"])
    .result.agg("count")
    .reset_index()
)

In [64]:
count_per_region_before_17

,region,vote,result
npartitions=1,,,
,category[unknown],category[unknown],int64
,...,...,...


In [65]:
count_per_region_before_17_computed = count_per_region_before_17.compute()

In [60]:
get_winner(count_per_region_before_17_computed)

winner
blue    297
red     241
Name: delegates, dtype: int64